In [ ]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import getpass
import os
import zipfile

In [ ]:
email = input("E-Mail: ")
password = getpass.getpass("Password: ")

users = {'Username':36} #username and number of pages in "search overview"
users

In [ ]:
# Create the payload
payload = {'userCredentialsForm.userCredentials.email':email,
          'userCredentialsForm.userCredentials.password':password,
           '_target1':'Login'
         }

In [ ]:
# Start the session
session = requests.Session()

In [ ]:
# Post the payload to the site to log in
s = session.get("https://old.sportstracklive.com/", data=payload)
s = session.get("https://old.sportstracklive.com/signin", data=payload)
s = session.post("https://old.sportstracklive.com/signin", data=payload)

In [ ]:
with open("login.html", 'wb') as f:
    print(f"Write file login.html")
    f.write(s.content)

In [ ]:
user_dashboard_url = f"http://old.sportstracklive.com/user/{list(users.keys())[0]}"
s = session.post(user_dashboard_url, data=payload)
with open("overview.html", 'wb') as f:
    print(f"Write file overview.html")
    f.write(s.content)

In [ ]:
data = dict()

for user, pages in users.items():
    print("User:", user)
    directory = os.path.join("website", user)

    if not os.path.exists(directory):
        os.makedirs(directory)
    
    pages = range(0,pages)
    print("pages:", list(pages))
    
           
    
    urls = []
    content = []
    for page in pages:
        url = f'http://old.sportstracklive.com/search?what=user:{user}&find=track&order=relevance&page={page}'
        
        urls.append(url)
         
        # Navigate to the next page and scrape the data
        s = session.get(url)

        content.append(s)

        filename = f"overview_{page}.html"
        path = os.path.join(directory, filename)

        with open(path, 'wb') as f:
            print(f"Write file {path}")
            f.write(s.content)
        
    print(urls)
    
    user_data = dict(
        urls = urls,
        content = content
    )
    
    data[user] = user_data

In [ ]:
data

In [ ]:
def save_html(url, directory, file_prefix):   
    
    s = session.get(url)
    filename = f"{file_prefix}.html"
    path = os.path.join(directory, filename)

    with open(path, 'wb') as f:
        #print(f"Write file {path}")
        f.write(s.content)

In [ ]:
def save_tracks(trackid, directory, file_prefix):
    
    exts = ['gpx'] #kml, csv possible as well
    
    for ext in exts:
    
        url = f"http://old.sportstracklive.com/track/{ext}?id={trackid}"
    
        #print(f"Download {url}")
        s = session.get(url)
        #print("Response:", s.ok)
            
        filename = f"{file_prefix}.{ext}.zip"
        path = os.path.join(directory, filename)

        if s.ok:
            with open(path, 'wb') as f:
                #print(f"Write file {path}")
                f.write(s.content)
        else:
            #print(f"No {ext}-data for {filename}")
            ...

In [ ]:
tracks = dict()

for user, user_data in data.items():
    print(user)
    directory = os.path.join("website", user, "tracks")

    if not os.path.exists(directory):
        os.makedirs(directory)
    
#    user_tracks = []
#    print(user_data['content'])
    for page in user_data['content']:

        #print(page.text)
        soup = BeautifulSoup(page.text, 'html.parser')
        infos = soup.find_all("a", title="Info")

        for info in infos:
            link = info.get('href')
            print(link)

            sports = link.split('/')[4]
            location = link.split('/')[5]
            trackid = link.rsplit('/',1)[1]

            #print(sports, location, trackid)


            file_prefix = f"{sports}-{location}-{trackid}"
            
            url = "http://old.sportstracklive.com/" + link
            save_html(url, directory, file_prefix)
            save_tracks(trackid, directory, file_prefix)
                
                
#    tracks[user] = user_tracks

In [ ]:
# Post-Processing (unzipping and deleting)
for user, user_data in data.items():
    print(user)
    directory = os.path.join("website", user, "tracks")
    
    directory_delete = os.path.join(directory, "delete")
    if not os.path.exists(directory_delete):
        os.makedirs(directory_delete)
        
    directory_gpx = os.path.join(directory, "routes")
    if not os.path.exists(directory_gpx):
        os.makedirs(directory_gpx)
    
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    for f in files:
        fp = os.path.join(directory, f)
        
        if fp.rsplit(".",1)[1] == 'zip':
            print("File:", fp)
            b = os.path.getsize(fp)
            print("Size:", b)
            
            if b < 1024:
                print("File too small -> delete")
                fp_delete = os.path.join(directory_delete, f)
                os.rename(fp, fp_delete)
                
            else:
                print("File size okay -> unzip")
                with zipfile.ZipFile(fp,"r") as zip_ref:
                    zip_ref.extractall(directory_gpx)
                    
                print("File extracted -> delete")
                fp_delete = os.path.join(directory_delete, f)
                os.rename(fp, fp_delete)